# Coursera Course - Applied Data Science Capstone  WK3

## Segmenting and Clustering Neighborhoods in Toronto

# Q1

In [7]:
import pandas as pd
import numpy as np
import requests

In [94]:
#Get content from wikipedia website
wiki_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki = requests.get(wiki_url).content

#Get the chart
df_list = pd.read_html(wiki)
df = df_list[0]

#save to csv file
df.to_csv('data.csv')

#Set header
headers = df.iloc[0]
df1  = pd.DataFrame(df.values[1:], columns=headers)

#Delete rows with Borough Not assigned
df1 = df1[df1.Borough != 'Not assigned']

#Replace neighborhood not assisgned with borough name
df1.Neighbourhood.replace('Not assigned',df1.Borough,inplace=True)

#Combine those sharing same postcode
df2 = df1.groupby('Postcode').agg(lambda x: ','.join(set(x))).reset_index()

print ('The shape of Toronto dataframe is',df2.shape)

df2.tail(10)


The shape of Toronto dataframe is (103, 3)


,Postcode,Borough,Neighbourhood
93,M9A,Etobicoke,Islington Avenue
94,M9B,Etobicoke,"West Deane Park,Cloverdale,Martin Grove,Isling..."
95,M9C,Etobicoke,"Bloordale Gardens,Eringate,Old Burnhamthorpe,M..."
96,M9L,North York,Humber Summit
97,M9M,North York,"Emery,Humberlea"
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Martin Grove Gardens,Richview Gardens,Kingsvie..."
101,M9V,Etobicoke,"Beaumond Heights,Albion Gardens,South Steeles,..."
102,M9W,Etobicoke,Northwest


# Q2

In [109]:
df3 = pd.read_csv('http://cocl.us/Geospatial_data')

df3.columns = ['Postcode','Latitude','Longitude']

df4 = df2.merge(df3, left_on='Postcode', right_on='Postcode',how = 'right')
df4.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Port Union,Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"West Hill,Guildwood,Morningside",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Q3

In [111]:
# import folium
!conda install -c conda-forge folium=0.5.0 --yes
import folium


Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  45.04 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  34.68 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  36.85 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  44.84 MB/s


In [144]:
#create map of toronto
trt_ltt = 43.6532; trt_lgt = -79.3832
map = folium.Map(location = [trt_ltt, trt_lgt], zoom_start = 10.7)

# add markers
for lat, lng, borough, neighborhood in zip(df4['Latitude'], df4['Longitude'], df4['Borough'], df4['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map)

#map

In [122]:

CLIENT_ID = 'FCDTS5TVB1FSLDELELRFS4Z10YUMRRHUD3OZAAOQRKN10GWW' # your Foursquare ID
CLIENT_SECRET = 'QGTK53IAWOYM1TTHENMFEMVSBM0U2OJCUSO5GLUOQOPXQM4L' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [162]:
Etobicoke_data = df4[df4['Borough'] == 'Etobicoke'].reset_index(drop=True)
print(Etobicoke_data.shape)
Etobicoke_data

(12, 5)


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M8V,Etobicoke,"Mimico South,Humber Bay Shores,New Toronto",43.605647,-79.501321
1,M8W,Etobicoke,"Long Branch,Alderwood",43.602414,-79.543484
2,M8X,Etobicoke,"Old Mill North,The Kingsway,Montgomery Road",43.653654,-79.506944
3,M8Y,Etobicoke,"King's Mill Park,Sunnylea,Mimico NE,The Queens...",43.636258,-79.498509
4,M8Z,Etobicoke,"Royal York South West,The Queensway West,South...",43.628841,-79.520999
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M9B,Etobicoke,"West Deane Park,Cloverdale,Martin Grove,Isling...",43.650943,-79.554724
7,M9C,Etobicoke,"Bloordale Gardens,Eringate,Old Burnhamthorpe,M...",43.643515,-79.577201
8,M9P,Etobicoke,Westmount,43.696319,-79.532242
9,M9R,Etobicoke,"Martin Grove Gardens,Richview Gardens,Kingsvie...",43.688905,-79.554724


In [116]:
#Create a map of Etobicoke and neighbor 
add_eto = 'Etobicoke,Toronto'
ltt_eto = 43.6205
lgt_eto = -79.5132
print('The geograpical coordinate of Etobicoke are {}, {}.'.format(ltt_eto, lgt_eto))

The geograpical coordinate of Etobicoke are 43.6205, -79.5132.


In [118]:
map_eto = folium.Map(location=[ltt_eto, lgt_eto], zoom_start=12)

# add markers to map
for lat, lng, label in zip(Etobicoke_data['Latitude'], Etobicoke_data['Longitude'], Etobicoke_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_eto)  
    
#map_eto

In [120]:
#Get top 100 venue in "westmount" in Etobicoke
nb_ltt = Etobicoke_data.loc[8, 'Latitude'] # neighbourhood latitude value
nb_lgt = Etobicoke_data.loc[8, 'Longitude'] # neighbourhood longitude value

nb_name = Etobicoke_data.loc[8, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of "{}" are {}, {}.'.format(nb_name, 
                                                               nb_ltt, 
                                                               nb_lgt))

Latitude and longitude values of "Westmount" are 43.696319, -79.53224240000002.


In [145]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, ltt_eto, lgt_eto, VERSION, radius, LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c7b44a09fb6b740ee1b969d'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4b0073a7f964a520643e22e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/movietheater_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d17f941735',
         'name': 'Movie Theater',
         'pluralName': 'Movie Theaters',
         'primary': True,
         'shortName': 'Movie Theater'}],
       'events': {'count': 28, 'summary': '28 movies'},
       'id': '4b0073a7f964a520643e22e3',
       'location': {'address': '1025 The Queensway',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'at Islington Ave.',
        'distance': 192,
        'formattedAddress': ['1025 The Queensway (a

In [125]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [126]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Cineplex Cinemas Queensway & VIP,Movie Theater,43.621251,-79.515346
1,Tim Hortons,Coffee Shop,43.620900,-79.515340
2,Goodwill,Thrift / Vintage Store,43.621602,-79.513198
3,Sausage Man,Food Truck,43.620165,-79.513815
4,Fogh Marine,Sporting Goods Shop,43.619122,-79.514810
5,Gardiner Expy at Islington,Intersection,43.620450,-79.513164
6,Cool Brewery,Brewery,43.618922,-79.510625
7,TTC Bus 15 Evans,Bus Line,43.618221,-79.511806


In [127]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

8 venues were returned by Foursquare.


In [128]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [129]:
#Get venues for each neighborhood in ETO
eto_venues = getNearbyVenues(names=Etobicoke_data['Neighbourhood'],
                                   latitudes=Etobicoke_data['Latitude'],
                                   longitudes=Etobicoke_data['Longitude']
                                  )

Mimico South,Humber Bay Shores,New Toronto
Long Branch,Alderwood
Old Mill North,The Kingsway,Montgomery Road
King's Mill Park,Sunnylea,Mimico NE,The Queensway East,Humber Bay,Old Mill South,Royal York South East,Kingsway Park South East
Royal York South West,The Queensway West,South of Bloor,Kingsway Park South West,Mimico NW
Islington Avenue
West Deane Park,Cloverdale,Martin Grove,Islington,Princess Gardens
Bloordale Gardens,Eringate,Old Burnhamthorpe,Markland Wood
Westmount
Martin Grove Gardens,Richview Gardens,Kingsview Village,St. Phillips
Beaumond Heights,Albion Gardens,South Steeles,Thistletown,Mount Olive,Silverstone,Jamestown,Humbergate
Northwest


In [164]:
eto_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Mimico South,Humber Bay Shores,New Toronto",43.605647,-79.501321,LCBO,43.602281,-79.499302,Liquor Store
1,"Mimico South,Humber Bay Shores,New Toronto",43.605647,-79.501321,New Toronto Fish & Chips,43.601849,-79.503281,Restaurant
2,"Mimico South,Humber Bay Shores,New Toronto",43.605647,-79.501321,Delicia Bakery & Pastry,43.601403,-79.503012,Bakery
3,"Mimico South,Humber Bay Shores,New Toronto",43.605647,-79.501321,Lucky Dice Restaurant,43.601392,-79.503056,Café
4,"Mimico South,Humber Bay Shores,New Toronto",43.605647,-79.501321,McDonald's,43.602470,-79.498963,Fast Food Restaurant
5,"Mimico South,Humber Bay Shores,New Toronto",43.605647,-79.501321,Subway,43.602382,-79.498275,Sandwich Place
6,"Mimico South,Humber Bay Shores,New Toronto",43.605647,-79.501321,Popeyes Louisiana Kitchen,43.602069,-79.499400,Fried Chicken Joint
7,"Mimico South,Humber Bay Shores,New Toronto",43.605647,-79.501321,Shoppers Drug Mart,43.601611,-79.502164,Pharmacy
8,"Mimico South,Humber Bay Shores,New Toronto",43.605647,-79.501321,Maple Leaf House,43.602040,-79.498678,American Restaurant
9,"Mimico South,Humber Bay Shores,New Toronto",43.605647,-79.501321,Coffee Time,43.602284,-79.499857,Coffee Shop


In [131]:
eto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Beaumond Heights,Albion Gardens,South Steeles,Thistletown,Mount Olive,Silverstone,Jamestown,Humbergate",10,10,10,10,10,10
"Bloordale Gardens,Eringate,Old Burnhamthorpe,Markland Wood",7,7,7,7,7,7
"King's Mill Park,Sunnylea,Mimico NE,The Queensway East,Humber Bay,Old Mill South,Royal York South East,Kingsway Park South East",1,1,1,1,1,1
"Long Branch,Alderwood",11,11,11,11,11,11
"Martin Grove Gardens,Richview Gardens,Kingsview Village,St. Phillips",4,4,4,4,4,4
"Mimico South,Humber Bay Shores,New Toronto",16,16,16,16,16,16
Northwest,3,3,3,3,3,3
"Old Mill North,The Kingsway,Montgomery Road",3,3,3,3,3,3
"Royal York South West,The Queensway West,South of Bloor,Kingsway Park South West,Mimico NW",14,14,14,14,14,14


In [132]:
print('There are {} uniques categories.'.format(len(eto_venues['Venue Category'].unique())))

There are 43 uniques categories.


In [133]:
# one hot encoding
eto_onehot = pd.get_dummies(eto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
eto_onehot['Neighborhood'] = eto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [eto_onehot.columns[-1]] + list(eto_onehot.columns[:-1])
eto_onehot = eto_onehot[fixed_columns]

eto_onehot.head(3)

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Beer Store,Burger Joint,Burrito Place,...,River,Sandwich Place,Seafood Restaurant,Shopping Plaza,Skating Rink,Smoke Shop,Supplement Shop,Tanning Salon,Thrift / Vintage Store,Wings Joint
0,"Mimico South,Humber Bay Shores,New Toronto",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Mimico South,Humber Bay Shores,New Toronto",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Mimico South,Humber Bay Shores,New Toronto",0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [134]:
eto_onehot.shape

(75, 44)

In [161]:
eto_grouped = eto_onehot.groupby('Neighborhood').mean().reset_index()
print(eto_grouped.shape)
eto_grouped.head(5)

(11, 44)


,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Beer Store,Burger Joint,Burrito Place,...,River,Sandwich Place,Seafood Restaurant,Shopping Plaza,Skating Rink,Smoke Shop,Supplement Shop,Tanning Salon,Thrift / Vintage Store,Wings Joint
0,"Beaumond Heights,Albion Gardens,South Steeles,...",0.0,0.000000,0.0,0.0,0.0,0.0,0.100000,0.0,0.0,...,0.0,0.100000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,"Bloordale Gardens,Eringate,Old Burnhamthorpe,M...",0.0,0.000000,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,...,0.0,0.000000,0.0,0.142857,0.000000,0.0,0.0,0.0,0.0,0.0
2,"King's Mill Park,Sunnylea,Mimico NE,The Queens...",0.0,0.000000,0.0,0.0,0.0,1.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,"Long Branch,Alderwood",0.0,0.090909,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.090909,0.0,0.000000,0.090909,0.0,0.0,0.0,0.0,0.0
4,"Martin Grove Gardens,Richview Gardens,Kingsvie...",0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [136]:
# Get Top 10 venues of each neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [166]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = eto_grouped['Neighborhood']

for ind in np.arange(eto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(eto_grouped.iloc[ind, :], num_top_venues)

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(11, 11)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Beaumond Heights,Albion Gardens,South Steeles,...",Grocery Store,Pizza Place,Fried Chicken Joint,Fast Food Restaurant,Beer Store,Sandwich Place,Coffee Shop,Pharmacy,Chinese Restaurant,Drugstore
1,"Bloordale Gardens,Eringate,Old Burnhamthorpe,M...",Liquor Store,Shopping Plaza,Convenience Store,Beer Store,Pharmacy,Pizza Place,Café,Wings Joint,Chinese Restaurant,Drugstore
2,"King's Mill Park,Sunnylea,Mimico NE,The Queens...",Baseball Field,Wings Joint,Café,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store,Dance Studio,Convenience Store,Coffee Shop
3,"Long Branch,Alderwood",Pizza Place,Gym,Sandwich Place,Dance Studio,Coffee Shop,Pharmacy,Pool,Pub,Skating Rink,Athletics & Sports
4,"Martin Grove Gardens,Richview Gardens,Kingsvie...",Mobile Phone Shop,Park,Pizza Place,Bus Line,Wings Joint,Chinese Restaurant,Fast Food Restaurant,Drugstore,Discount Store,Dance Studio
5,"Mimico South,Humber Bay Shores,New Toronto",Café,Gym,Pizza Place,Bakery,Coffee Shop,Fast Food Restaurant,Flower Shop,Fried Chicken Joint,Liquor Store,Mexican Restaurant
6,Northwest,Rental Car Location,Drugstore,Bar,Café,Flower Shop,Fast Food Restaurant,Discount Store,Dance Studio,Convenience Store,Coffee Shop
7,"Old Mill North,The Kingsway,Montgomery Road",Smoke Shop,Park,River,Wings Joint,Café,Fast Food Restaurant,Drugstore,Discount Store,Dance Studio,Convenience Store
8,"Royal York South West,The Queensway West,South...",Wings Joint,Thrift / Vintage Store,Bakery,Burger Joint,Burrito Place,Convenience Store,Discount Store,Fast Food Restaurant,Flower Shop,Grocery Store
9,"West Deane Park,Cloverdale,Martin Grove,Isling...",Bank,Wings Joint,Café,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store,Dance Studio,Convenience Store,Coffee Shop


In [178]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

eto_grouped_clustering = eto_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(eto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 3, 1, 1, 1, 4, 2, 1, 0], dtype=int32)

In [182]:
eto_merged1 = Etobicoke_data

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
eto_merged1 = eto_merged1.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

eto_merged1.head() # check the last columns!

#print(neighborhoods_venues_sorted.shape)

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M8V,Etobicoke,"Mimico South,Humber Bay Shores,New Toronto",43.605647,-79.501321,1.0,Café,Gym,Pizza Place,Bakery,Coffee Shop,Fast Food Restaurant,Flower Shop,Fried Chicken Joint,Liquor Store,Mexican Restaurant
1,M8W,Etobicoke,"Long Branch,Alderwood",43.602414,-79.543484,1.0,Pizza Place,Gym,Sandwich Place,Dance Studio,Coffee Shop,Pharmacy,Pool,Pub,Skating Rink,Athletics & Sports
2,M8X,Etobicoke,"Old Mill North,The Kingsway,Montgomery Road",43.653654,-79.506944,2.0,Smoke Shop,Park,River,Wings Joint,Café,Fast Food Restaurant,Drugstore,Discount Store,Dance Studio,Convenience Store
3,M8Y,Etobicoke,"King's Mill Park,Sunnylea,Mimico NE,The Queens...",43.636258,-79.498509,3.0,Baseball Field,Wings Joint,Café,Flower Shop,Fast Food Restaurant,Drugstore,Discount Store,Dance Studio,Convenience Store,Coffee Shop
4,M8Z,Etobicoke,"Royal York South West,The Queensway West,South...",43.628841,-79.520999,1.0,Wings Joint,Thrift / Vintage Store,Bakery,Burger Joint,Burrito Place,Convenience Store,Discount Store,Fast Food Restaurant,Flower Shop,Grocery Store


In [186]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[ltt_eto, lgt_eto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(eto_merged1['Latitude'], eto_merged1['Longitude'], eto_merged1['Neighbourhood'], eto_merged1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

TypeError: list indices must be integers or slices, not float